In [ ]:
import pandas as pd
import json
import os
import datetime
import shutil

* The aim of this notebook is to acquire the images from our dataset that have corresponding ground truths and save them to a new file.
* These images will then be used to test the model architecture previously developed

<b> Saving INESC plants ground truths in a JSON file </b>

In [ ]:
biometric_df = pd.read_csv("biom_mod.csv", delimiter=';')
biometric_df.drop(columns=["Thickness 1", "Thickness 2", "Thickness 3", "Thickness 4", "Thickness 5", "Line", "Sample", "Number", "Perpendicular", "No leaves"], inplace=True)

In [ ]:
biometric_df["Date"] = pd.to_datetime(biometric_df["Date"], dayfirst=True)

In [ ]:
datetime.datetime.strptime("21/12/2008", "%d/%m/%Y").strftime("%Y-%m-%d")
#biometric_df["Date"] = biometric_df["Date"].replace("/", "-")

In [ ]:
biometric_df

In [ ]:
biometric_df["Date"].dtype

In [ ]:
biometry = {}

In [ ]:
for index, row in biometric_df.iterrows():
    date = str(row["Date"])[:10]
    print(date)
    image_name=row["CODE"]+"_"+date
    biometry[image_name] = {
        "Date": date,
        "Plant": int(row["CODE"]),
        "Diameter": int(row["Diameter"]),
        "Height": int(row["Height"])
    }

In [ ]:
# Save the biometry ground truths to a JSON file
with open("../../GroundTruth_INESC/biometry_inesc_gt.json", "w") as json_file:
    json.dump(biometry, json_file, indent=4)

<b> Rename the image files to coincide with the keys of the ground truths values </b>

In [ ]:
images = os.listdir("../../NewRGBImages/raw")

In [ ]:
import re
def format_date_string(s):
    # Remove trailing letter if present
    cleaned = re.sub(r'[A-Za-z]$', '', s)
    
    # Split on dashes and pad the last part (day) if needed
    parts = cleaned.split('-')
    if len(parts) >= 3:
        parts[-1] = parts[-1].zfill(2)
        return '-'.join(parts)
    return cleaned

In [ ]:
for file_name in images:
    old_file = file_name
    new_file_name= old_file[8:25]
    new_file_name = format_date_string(new_file_name)
    new_file_name = new_file_name+".png"
    print(new_file_name)
    old_file = os.path.join("../../NewRGBImages/raw", old_file)
    new_file = os.path.join("../../NewRGBImages/formated_name", new_file_name)
    os.rename(old_file, new_file)

<b> Check INESC image files that have a corresponding Ground Truth </b>

In [ ]:
inesc_images = os.listdir("../../NewRGBImages/formated_name")

In [ ]:
with open('../../GroundTruth_INESC/biometry_inesc_gt.json', 'r') as file:
    ground_truths_inesc = json.load(file)

In [ ]:
# Images with a corresponding ground truth
# Move files to new directory
og_dir = "../../NewRGBImages/formated_name/"
new_dir = "../../NewRGBImages/images_with_gt/"
counter = 0
images_with_ground_t = []
for img in inesc_images:
    image_name = img[:17]
    if image_name in ground_truths_inesc:
        print(f"{img} has a corresponding Ground Truth\n")
        images_with_ground_t.append(img)
        shutil.move(og_dir+img, new_dir)
        counter+=1
        print("--------------------")
    else:
        print(f"No correspondence for {img}\n")
        print("--------------------")

        continue
        
print(counter)

In [ ]:
images_with_ground_t

In [ ]:
images_with_gt = os.listdir("../../NewRGBImages/images_with_gt")

In [ ]:
len(images_with_ground_t)